# COMMON FUNCTIONS

In [1]:
def float_bin(my_number, places = 3):
    my_whole, my_dec = str(my_number).split(".")
    my_whole = int(my_whole)
    res = (str(bin(my_whole))+".").replace('0b','')
    
    for x in range(places):
        my_dec = str('0.')+str(my_dec)
        temp = '%1.20f' %(float(my_dec)*2)
        my_whole, my_dec = temp.split(".")
        res += my_whole
    return res
 
def bin_float(my_str):
    str_split = my_str.split('.')
    whole_str = str_split[0]
    dec_str = str_split[1]
    
    res = int(whole_str,2)
    for x in range(len(dec_str)):
        if(dec_str[x]=='1'):
            res += 2**(-(x+1))
    return res
 
def float16_bin_IEEE754(n) :
    # identifying whether the number
    # is positive or negative
    sign = 0
    if n < 0 :
        sign = 1
        n = n * (-1)
    p = 30
    # convert float to binary
    dec = float_bin (n, places = p)

    dotPlace = dec.find('.')
    onePlace = dec.find('1')
    # finding the mantissa
    is_zero = False
    if onePlace == -1:
        is_zero = True
    elif onePlace > dotPlace:
        dec = dec.replace(".","")
        onePlace -= 1
        dotPlace -= 1
    elif onePlace < dotPlace:
        dec = dec.replace(".","")
        dotPlace -= 1
    mantissa = dec[onePlace+1:]
 
    # calculating the exponent(E)
    exponent = dotPlace - onePlace
    exponent_bits = exponent + 15
 
    # converting the exponent from
    # decimal to binary
    exponent_bits = bin(exponent_bits).replace("0b",'')
 
    # the IEEE754 notation in binary    
    if is_zero == True:
        final = '0'*16
    else:
        final = str(sign) + exponent_bits.zfill(5) + mantissa[0:10]
 
    # convert the binary to hexadecimal
    hstr = '0x%0*X' %((len(final) + 3) // 4, int(final, 2))
    return (hstr, final)

def bin_float16_IEEE754(bin):
    sign = (-1)**int(bin[0])
    exponent = int(bin[1:6],2)-15
    fraction = 0.0
    for i in range(6,16):
        fraction = fraction + int(bin[i])*(2**(5-i))
    res = sign*(1+fraction)*(2**exponent)
    return res



(hex16,bin16) = float16_bin_IEEE754(9.132)
float16 = bin_float16_IEEE754(bin16)
print (hex16)
print (float16)

print('-------------------')
bin_val = float_bin(51.329, 8)
print(bin_val)
print(bin_float(bin_val))


0x4890
9.125
-------------------
110011.01010100
51.328125


In [2]:
import numpy as np
def split(a, n):
    k, m = divmod(len(a), n)
    return (a[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n))
def tree_bram_gen(dtree_list, no_parts, tree_bram):
    div_tree_list = list(split(dtree_list,no_parts))
    
    absolute_idx = 0
    for dtp_tree in div_tree_list:
        for tree in dtp_tree:
            if dtp_tree.index(tree) == 0:
                absolute_addr = no_parts + len(tree_bram) - absolute_idx
                if absolute_idx == no_parts-1:
                    absolute_addr_bin = ("{0:0"+str(BRAM_DWIDTH-1)+"b}").format(absolute_addr) + '1'
                else:
                    absolute_addr_bin = ("{0:0"+str(BRAM_DWIDTH-1)+"b}").format(absolute_addr) + '0'
                tree_bram.insert(absolute_idx,absolute_addr_bin)
                absolute_idx = absolute_idx + 1
                
            dtree = tree.tree_
            is_last = (tree == dtp_tree[-1])
            for node_idx in range(dtree.node_count):
                is_leaf = dtree.children_left[node_idx] == dtree.children_right[node_idx]

                if bool(is_leaf) is True:
                    if IS_CLF:
                        result = np.argmax(dtree.value[node_idx]).item()
                    else:
                        result = int(dtree.value[node_idx][0][0].item() * TO_RANGE / FROM_RANGE)
                    next_tree = dtree.node_count - node_idx

                    result_bin = ("{0:0"+str(RES_WIDTH)+"b}").format(result)
                    next_tree_bin = ("{0:0"+str(TREE_WIDTH)+"b}").format(next_tree)
                    is_leaf_bin = "{0:01b}".format(int(is_leaf))
                    is_last_bin = "{0:01b}".format(int(is_last))

                    bram_node = next_tree_bin[0:TREE_WIDTH] + result_bin[0:RES_WIDTH] + is_last_bin[0] + is_leaf_bin[0]
                else:
                    right_child_addr = dtree.children_right[node_idx].item() - node_idx - 1
                    threshold = dtree.threshold[node_idx].item()
                    attribute = dtree.feature[node_idx].item()

                    right_child_addr_bin = ("{0:0"+str(RCHLD_WIDTH)+"b}").format(right_child_addr)
                    
                    if IS_INT:
                        threshold_bin = ("{0:0"+str(THSH_WIDTH)+"b}").format(round(threshold))
                    else:
                        (_ ,threshold_bin) = float16_bin_IEEE754(threshold)
                        
                    attribute_bin = ("{0:0"+str(ATTR_WIDTH)+"b}").format(attribute)
                    is_leaf_bin = "{0:01b}".format(int(is_leaf))

                    bram_node = right_child_addr_bin[0:RCHLD_WIDTH] + attribute_bin[0:ATTR_WIDTH] + threshold_bin[0:THSH_WIDTH] + is_leaf_bin[0]
                tree_bram.append(bram_node)

In [3]:
def rambin2ramdec(rambin, no_parts, ramdec):
    node_addr = 0
    for node in rambin:
        node_dict = {}
        if rambin.index(node) < no_parts: # these node are absolute address
            node_dict['is_last_absolute_addr'] = bool(int(node[(BRAM_DWIDTH-1)]))
            node_dict['absolute_addr'] = int(node[0:(BRAM_DWIDTH-1)],2)
        elif int(node[BRAM_DWIDTH-1]) == 1:
            node_dict['is_leaf'] = True
            node_dict['is_last'] = bool(int(node[(BRAM_DWIDTH-1)-LAST_IDX]))
            node_dict['result'] = int(node[(BRAM_DWIDTH-1)-(RES_IDX+RES_WIDTH-1):(BRAM_DWIDTH-1)-RES_IDX+1],2)
            node_dict['next_tree'] = int(node[(BRAM_DWIDTH-1)-(TREE_IDX+TREE_WIDTH-1):(BRAM_DWIDTH-1)-TREE_IDX+1],2)
        else:
            node_dict['is_leaf'] = False
            node_dict['right_child'] = int(node[(BRAM_DWIDTH-1)-(RCHLD_IDX+RCHLD_WIDTH-1):(BRAM_DWIDTH-1)-RCHLD_IDX+1],2)
            thres = node[(BRAM_DWIDTH-1)-(THSH_IDX+THSH_WIDTH-1):(BRAM_DWIDTH-1)-THSH_IDX+1]
            #node_dict['threshold'] = '0x%0*X' %((len(thres) + 3) // 4, int(thres, 2))
            #node_dict['threshold'] += ' - ' +  str(bin_float16_IEEE754(thres))
            
            if IS_INT:
                node_dict['threshold'] = int(thres, 2)
            else:
                node_dict['threshold'] = bin_float16_IEEE754(thres)
            
            node_dict['attribute'] = int(node[(BRAM_DWIDTH-1)-(ATTR_IDX+ATTR_WIDTH-1):(BRAM_DWIDTH-1)-ATTR_IDX+1],2)
        ramdec.append(node_dict)
def print_dec_ram(dec_ram):
    tree_idx = 0
    dtp_idx = 0
    print(("Tree "+str(tree_idx)).center(120, "-"))
    print(("PART "+str(dtp_idx)).center(120, "*"))
    for i in range(len(dec_ram)):
        print("addr ",i,": ", dec_ram[i])

        if 'is_leaf' in dec_ram[i]:
            if dec_ram[i]['is_leaf'] == True:
                if dec_ram[i]['next_tree'] == 1:
                    tree_idx = tree_idx+1
                    print(("Tree "+str(tree_idx)).center(120, "-"))
                    if dec_ram[i]['is_last'] == True:
                        dtp_idx = dtp_idx+1
                        print(("PART "+str(dtp_idx)).center(120, "*"))

In [4]:
def sample_dec2bin(sample):
    sample_bin = []
    for sample in sample.values:
        attr_list = []
        for attr_dec in sample:
            if IS_INT:
                attr_bin = ("{0:0"+str(16)+"b}").format(round(attr_dec.item()))
            else:
                (_,attr_bin) = float16_bin_IEEE754(attr_dec.item())
            attr_list.append(attr_bin)
        sample_bin.append(attr_list)
    return sample_bin

In [5]:
def tree_coeff_file_gen(rambin, filename):
    f = open(filename+'.coe', "w")
    f.write("memory_initialization_radix=16;\n")
    f.write("memory_initialization_vector=")
    for node in rambin:
        hex_str = ('0x%0*X' %((len(node) + 3) // 4, int(node, 2)))+' '
#         print(hex_str)
        f.write(hex_str[2:None])
    f.write(";\n")
    f.close()
def attribute_coeff_file_gen(samples, filename):
    f = open(filename+'.coe', "w")
    f.write("memory_initialization_radix=16;\n")
    f.write("memory_initialization_vector=")
    
    for s in samples:
        for a in s:
            if IS_INT:
                attr_hex = ("{0:0"+str(4)+"X}").format(round(a.item()))
            else:
                (attr_hex, _) = float16_bin_IEEE754(a.item())
                attr_hex = attr_hex[2:None]
            f.write(attr_hex+' ')
    f.write(';')
    f.close()
def sample_mem_file_gen(samples, filename):
    f = open(filename+'.txt', "w")
    
    for s in samples:
        for a in s:
            f.write(str(int(a, 2))+' ')
        f.write('\n')
    
    f.close()
def tree_mem_file_gen(dtp_rams, filename):
    f = open(filename+'.txt', "w")
    
    for dtp in dtp_rams:
        for n in dtp:
            f.write(str(int(n, 2))+' ')
        f.write('\n')
    
    f.close()

# REGRESSION

In [1]:
#Import scikit-learn dataset library
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()

In [3]:
print(housing.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

print(housing.target_names)

print(housing.feature_names)

In [5]:
# print the iris data (top 5 records)
print(housing.data[0:5])

# print the iris labels (0:setosa, 1:versicolor, 2:virginica)
print(housing.target)

[[ 8.32520000e+00  4.10000000e+01  6.98412698e+00  1.02380952e+00
   3.22000000e+02  2.55555556e+00  3.78800000e+01 -1.22230000e+02]
 [ 8.30140000e+00  2.10000000e+01  6.23813708e+00  9.71880492e-01
   2.40100000e+03  2.10984183e+00  3.78600000e+01 -1.22220000e+02]
 [ 7.25740000e+00  5.20000000e+01  8.28813559e+00  1.07344633e+00
   4.96000000e+02  2.80225989e+00  3.78500000e+01 -1.22240000e+02]
 [ 5.64310000e+00  5.20000000e+01  5.81735160e+00  1.07305936e+00
   5.58000000e+02  2.54794521e+00  3.78500000e+01 -1.22250000e+02]
 [ 3.84620000e+00  5.20000000e+01  6.28185328e+00  1.08108108e+00
   5.65000000e+02  2.18146718e+00  3.78500000e+01 -1.22250000e+02]]
[4.526 3.585 3.521 ... 0.923 0.847 0.894]


In [6]:
# Creating a DataFrame of given iris dataset.
import pandas as pd
data=pd.DataFrame({
    'MedInc':housing.data[:,0],
    'HouseAge':housing.data[:,1],
    'AveRooms':housing.data[:,2],
    'AveBedrms':housing.data[:,3],
    'Population':housing.data[:,4],
    'AveOccup':housing.data[:,5],
    'Latitude':housing.data[:,6],
    'Longitude':housing.data[:,7],
    'price':housing.target
})
data.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,price
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [7]:
# Import train_test_split function
from sklearn.model_selection import train_test_split

X=data[['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']]  # Features
y=data['price']  # Labels

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70% training and 30% test

In [35]:
X_train_file = 'X_train.csv'
X_test_file  = 'X_test.csv'
y_train_file = 'y_train.csv'
y_test_file  = 'y_test.csv'

In [34]:
# save dataframe to csv
X_train.to_csv(X_train_file, sep='\t', encoding='utf-8', index=False)
X_test.to_csv(X_test_file, sep='\t', encoding='utf-8', index=False)
y_train.to_csv(y_train_file, sep='\t', encoding='utf-8', index=False)
y_test.to_csv(y_test_file, sep='\t', encoding='utf-8', index=False)

# DATASET LOADING

In [6]:
# load csv to dataframe
import pandas as pd
X_train = pd.read_csv('X_train.csv', sep='\t', encoding='utf-8')
X_test  = pd.read_csv('X_test.csv', sep='\t', encoding='utf-8')
y_train = pd.read_csv('y_train.csv', sep='\t', encoding='utf-8')
y_test  = pd.read_csv('y_test.csv', sep='\t', encoding='utf-8')

In [7]:
from IPython.display import display, HTML

display(X_train[0:5])
display(X_test[0:5])
display(y_train[0:5])
display(y_test[0:5])

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,3.8125,14.0,5.941839,1.033771,1787.0,3.352720,38.65,-121.46
1,3.0100,26.0,4.010165,1.088945,2394.0,3.041931,33.93,-118.35
2,4.3118,20.0,5.764849,0.994304,3455.0,2.811229,38.68,-121.35
3,1.7589,28.0,3.502874,1.172414,1816.0,5.218391,33.79,-118.17
4,2.4632,24.0,2.747039,1.083756,2642.0,2.235195,34.07,-118.28


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,3.5967,15.0,4.257391,1.046957,1666.0,2.897391,33.75,-117.93
1,3.0253,29.0,5.673260,1.007326,3316.0,2.429304,39.75,-121.82
2,6.3669,47.0,6.520270,1.064189,692.0,2.337838,37.91,-122.27
3,2.9119,45.0,4.796020,0.860697,613.0,3.049751,37.72,-122.45
4,6.9508,18.0,6.906578,0.980934,2893.0,2.757865,37.22,-121.86


,price
0,0.926
1,1.919
2,1.200
3,1.183
4,2.167


,price
0,1.416
1,1.114
2,3.155
3,2.289
4,3.612


In [8]:
min(y_test.values)

array([0.14999])

In [40]:
# #Import Random Forest Model
# from sklearn.ensemble import RandomForestRegressor

# rgs=RandomForestRegressor(n_estimators=100, max_depth = 9)

# #Train the model using the training sets y_pred=clf.predict(X_test)
# rgs.fit(X_train,y_train.values.ravel())

In [41]:
# import joblib

# # save
# joblib.dump(rgs, "random_forest_rgs.joblib")

# MODEL LOADING

In [9]:
import joblib
rgs = joblib.load("./random_forest_rgs.joblib")

In [10]:
import time

exec_time_list = []
accum = 0
for i in range(20):
    start_time = time.time()
    rgs.predict(X_test[0:1000])
    end_time = time.time()
    exec_time = end_time - start_time
    print(exec_time)
    accum += exec_time
    exec_time_list.append(exec_time)
print('mean:',accum/len(exec_time_list))

0.03487253189086914
0.025048255920410156
0.026922225952148438
0.027039289474487305
0.0280914306640625
0.026966571807861328
0.025111675262451172
0.024994373321533203
0.02909064292907715
0.02593088150024414
0.026927709579467773
0.026927947998046875
0.026025056838989258
0.026050567626953125
0.029022693634033203
0.02803635597229004
0.026027441024780273
0.030916929244995117
0.03789949417114258
0.025931119918823242
mean: 0.027891659736633302


In [11]:
from sklearn.metrics import mean_squared_error
import numpy as np

y_pred=rgs.predict(X_test[0:1000])
mse = mean_squared_error(y_test[0:1000], y_pred)
rmse = mse**.5
print(mse)
print(rmse)

0.3262003702367501
0.5711395365729377


In [12]:
print(rgs.predict(X_test))

[2.00299976 1.0271564  3.78167639 ... 4.71339219 4.93292265 1.08348944]


In [13]:
import numpy as np
from numpy import interp

def predict_map_int(tree, sample, from_lb,from_ub, to_lb,to_ub):
    accum = np.array([0]*len(sample))
    for t in tree.estimators_:
        predict = np.rint(interp(t.predict(sample.values),[from_lb,from_ub],[to_lb,to_ub]))
        accum = np.add(accum, predict)
    return interp(accum / len(tree.estimators_), [to_lb,to_ub],[from_lb,from_ub])

y_int = predict_map_int(rgs, X_test, 0,5,0,100)
print(y_int)

mse_int = mean_squared_error(y_test, y_int)
rmse_int = mse_int**.5
print(mse_int)
print(rmse_int)

[2.005  1.025  3.781  ... 4.715  4.9385 1.0845]
0.322105675773498
0.5675435452663505


In [14]:
# percentage of different
def percent_dif(x1,x2):
    return abs(x1-x2)/abs((x1+x2)/2)

print(percent_dif(rmse,rmse_int))

0.006316052928052017


In [15]:
FROM_RANGE       = 5
TO_RANGE         = 100

def print_tree(tree):
    for i in range(len(tree.estimators_)):
        print("Tree",i,'-----------------------------------------------------------------------------------')
        print("Number of nodes: ", tree.estimators_[i].tree_.node_count) #Number of node
#         print("array of left children: ", tree.estimators_[i].tree_.children_left) #array of left children
#         print("array of right children: ", tree.estimators_[i].tree_.children_right) #array of right children
#         print("array of nodes splitting feature: ", tree.estimators_[i].tree_.feature) #array of nodes splitting feature
#         print("array of nodes splitting points: ", tree.estimators_[i].tree_.threshold) #array of nodes splitting points
#         for n_i in range(tree.estimators_[i].tree_.node_count):
#             if(tree.estimators_[i].tree_.children_left[n_i] == tree.estimators_[i].tree_.children_right[n_i]):
#                 print("array of nodes values: ", int(tree.estimators_[i].tree_.value[n_i][0][0].item() * TO_RANGE / FROM_RANGE)) #array of nodes values

In [16]:
print_tree(rgs)

Tree 0 -----------------------------------------------------------------------------------
Number of nodes:  817
Tree 1 -----------------------------------------------------------------------------------
Number of nodes:  791
Tree 2 -----------------------------------------------------------------------------------
Number of nodes:  797
Tree 3 -----------------------------------------------------------------------------------
Number of nodes:  737
Tree 4 -----------------------------------------------------------------------------------
Number of nodes:  761
Tree 5 -----------------------------------------------------------------------------------
Number of nodes:  743
Tree 6 -----------------------------------------------------------------------------------
Number of nodes:  803
Tree 7 -----------------------------------------------------------------------------------
Number of nodes:  745
Tree 8 -----------------------------------------------------------------------------------
Numbe

# BIN RAM GEN

In [17]:
BRAM_DWIDTH      = 32
BRAM_AWIDTH      = 13

TREE_DEPTH       = 10 # number of levels
LEAF_IDX         = 0
THSH_IDX         = 1
THSH_WIDTH       = 16
ATTR_IDX         = 17
ATTR_WIDTH       = 5
RCHLD_IDX        = 22
RCHLD_WIDTH      = BRAM_DWIDTH - ATTR_WIDTH - THSH_WIDTH - 1
LAST_IDX         = 1
RES_IDX          = 2
RES_WIDTH        = 16
TREE_IDX         = 18
TREE_WIDTH       = BRAM_DWIDTH - RES_WIDTH - 2

LAST_AB_IDX      = 0
AB_TREE_IDX      = 1
AB_TREE_WIDTH    = BRAM_AWIDTH

IS_INT = False
IS_CLF = False

In [25]:
N_ESTIMATORS = 100
N_DTPS = 15
PIPELINE_STAGES = 5

bin_tree_rams = [] # list of Tree RAMs
dtp_trees = list(split(rgs.estimators_, N_DTPS)) # list of subset of tree (Each subset belongs to each DTP)
for subset in dtp_trees:
    bin_bram = []
    tree_bram_gen(dtree_list=subset, no_parts=PIPELINE_STAGES, tree_bram=bin_bram)
    bin_tree_rams.append(bin_bram)

In [26]:
for bin_tree_ram in bin_tree_rams:
    print(len(bin_tree_ram))

5454
5462
5594
5662
5536
5412
5530
5554
5522
5710
4643
4793
4833
4773
4935


In [27]:
bin_tree_rams[0]

['00000000000000000000000000001010',
 '00000000000000000000110010011010',
 '00000000000000000001100010010110',
 '00000000000000000001111010001000',
 '00000000000000000010010001010111',
 '01110001010000001000100111111100',
 '00111000010000001000001101011100',
 '00011011110001001000011110111110',
 '00001110110010101000010001110000',
 '00000111010000001000000010111110',
 '00000011010001001000001000111100',
 '00000001010010001011010111100000',
 '00000000110011111010111001111100',
 '00000000010000000111101100101010',
 '00001100101000000000000010000001',
 '00001100100111000000000001010001',
 '00001100100110000000000000010101',
 '00000000110000101001111000000000',
 '00000000010011111010111011000110',
 '00001100100011000000000010001101',
 '00001100100010000000000011011001',
 '00000000010010100111110011000000',
 '00001100100000000000000010000101',
 '00001100011111000000000100011101',
 '00000001110001001000010101101010',
 '00000000110000000111101000010100',
 '00000000010010100111110011111100',
 

# DEC RAM GEN

In [28]:
dec_tree_rams = []
for bin_tree_ram in bin_tree_rams:
    dec_tree_ram = []
    rambin2ramdec(bin_tree_ram, PIPELINE_STAGES, dec_tree_ram)
    dec_tree_rams.append(dec_tree_ram)

# BIN SAM GEN

In [29]:
N_SAMPLES = 1000
bin_sample = sample_dec2bin(X_test[0:N_SAMPLES]) # list of samples in binary

# for i in range(len(bin_sample)):
#     print(i,end=' ')
#     for a in bin_sample[i]:
#         print(a, end=' ')
#     print('')

# PREDICT

In [30]:
def decision_path(sample, ramdec, start_idx):
    accum = 0
    idx = start_idx
    while True:
#         print(idx) # for decision path
        if ramdec[idx]['is_leaf'] == True:
            accum += ramdec[idx]['result']
#             print('result:', ramdec[idx]['result'])
            if ramdec[idx]['is_last'] ==  True: 
                print(idx)
                break
            idx += ramdec[idx]['next_tree']
        else:
            dec_attr = int(sample[ramdec[idx]['attribute']],2) if IS_INT else bin_float16_IEEE754(sample[ramdec[idx]['attribute']])
            
            if dec_attr <= ramdec[idx]['threshold']:
                idx += 1
            else:
                idx += ramdec[idx]['right_child'] + 1
    return accum

def decision_path_bram(sample_list, ramdec_list):
    accums = []
    for s_idx in range(len(sample_list)):
#         print('Sample',s_idx,'****************************************************************************') # for decision path
        accum = 0
        for d_idx in range(len(ramdec_list)):
#             print('DTP',d_idx,'****************************************************************************')
            for node_idx in range(PIPELINE_STAGES):
#                 print('Stage', node_idx, '------------------------------------------') # for decision path
                accum += decision_path(sample_list[s_idx], ramdec_list[d_idx], int(ramdec_list[d_idx][node_idx]['absolute_addr']))
                if ramdec_list[d_idx][node_idx]['is_last_absolute_addr'] == True:
                    break
        accums.append(accum)
    return accums

accum_res = decision_path_bram(bin_sample, dec_tree_rams)

1173
2750
3482
4239
5035
1104
2680
3531
4298
5073
1094
2739
3519
4367
5177
1130
2776
3593
4396
5205
1164
2822
3603
4376
5144
1147
2726
3473
4211
4967
1107
2760
3574
4344
5127
1140
2770
3549
4360
5114
1117
2664
3475
4289
5063
1180
2826
3647
4448
5256
1118
1876
2678
3400
4194
1133
1935
2712
3506
4331
1216
1995
2774
3564
4366
1196
1986
2792
3578
4395
1235
2063
2899
3683
4472
1039
2600
3354
4098
4863
973
2512
3354
4138
4928
951
2756
3539
4389
5001
983
2645
3456
4210
5240
1024
2629
3459
4219
5002
969
2573
3316
4053
4821
969
2594
3432
4202
4978
986
2632
3394
4208
4969
985
2489
3334
4142
4910
1027
2664
3512
4307
5130
975
1754
2537
3266
4066
1007
1762
2576
3358
4173
1014
1861
2624
3424
4222
1035
1850
2644
3423
4212
1041
1926
2747
3543
4357
1381
2904
3702
4432
5147
1309
2860
3713
4466
5313
1291
2891
3754
4596
5338
1340
2959
3792
4589
5400
1374
2993
3791
4551
5330
1377
2908
3630
4469
5147
1315
2963
3774
4555
5299
1346
2974
3815
4536
5309
1320
2915
3681
4436
5230
1364
3018
3858
4628
5479
1313
205

1052
1930
2754
3544
4357
1131
2678
3422
4196
4945
1049
2619
3440
4243
5028
1047
2635
3432
4288
5127
1097
2707
3502
4328
5146
1125
2763
3555
4293
5068
1075
2673
3402
4131
4901
1064
2701
3499
4299
5097
1090
2713
3504
4308
5082
1056
2601
3418
4238
4974
1129
2760
3590
4397
5228
1054
1821
2616
3334
4145
1105
1874
2405
3435
4266
1149
1941
2701
3492
4323
1151
1932
2728
3540
4309
1111
2033
2847
3626
4439
922
2486
3234
4105
4744
835
2532
3224
4145
4792
799
2441
3267
4101
4849
989
2505
3338
4225
4939
911
2649
3324
4120
5011
979
2580
3324
3949
4706
853
2613
3306
4070
4988
871
2515
3400
4220
4980
864
2498
3206
3975
4784
914
2556
3362
4173
5010
887
1610
2400
3148
3950
879
1774
2496
3222
4041
1031
1726
2454
3248
4094
1046
1714
2516
3440
4223
1051
1788
2597
3419
4225
977
2526
3287
4083
4803
903
2505
3287
4130
4864
896
2495
3331
4163
4969
967
2583
3398
4179
5048
965
2595
3402
4155
4986
954
2560
3298
4003
4771
911
2585
3375
4157
4956
977
2578
3380
4193
4956
927
2473
3284
4092
4846
976
2629
3427
4230
50

5462
1080
1857
2634
3608
4378
1324
1879
2683
3450
4522
1359
1981
2939
3744
4549
1156
1945
2980
3764
4552
1362
2251
2854
3632
4446
1016
2525
3286
4082
4825
954
2361
3334
3995
4879
881
2537
3353
4162
4949
882
2554
3416
4169
4975
1007
2590
3420
4188
4983
806
2411
3144
4022
4770
930
2583
3368
4193
4959
974
2539
3218
4048
4792
938
2356
3302
4058
4871
952
2616
3426
4229
5060
947
1678
2497
3200
4030
973
1636
2371
3320
4100
978
1823
2587
3352
4147
1019
1774
2577
3385
4194
1022
1886
2708
3507
4316
1266
2802
3565
4313
5060
1189
2739
3548
4338
5126
1345
2905
3719
4580
5256
1325
2840
3655
4628
5381
1384
2871
3661
4567
5229
1209
2782
3533
4266
5165
1330
2841
3637
4596
5215
1396
2837
3613
4428
5193
1199
2716
3707
4354
5255
1390
2876
3723
4503
5493
1165
1949
2757
3643
4249
1328
1987
2782
3576
4563
1268
2060
2828
3776
4606
1260
2100
3014
3656
4439
1419
2299
2951
3732
4532
1290
2845
3590
4328
5112
1211
2770
3581
4374
5166
1355
2952
3809
4616
5256
1387
2858
3684
4651
5448
1415
2921
3707
4613
5253
1245
2

3111
3847
4644
5408
1338
2975
3744
4419
5227
1397
3047
3855
4645
5396
1515
3064
3776
4607
5375
1387
2859
3847
4581
5407
1525
3106
3932
4760
5577
1373
2148
2937
3771
4529
1438
2191
3049
3807
4689
1489
2212
3031
3899
4700
1469
2315
3161
3846
4639
1574
2432
3092
3997
4753
1088
2652
3376
4131
4899
1033
2582
3401
4172
4939
1241
2828
3659
4529
5282
1275
2892
3733
4526
5328
1289
2720
3512
4479
5042
1046
2638
3597
4100
5097
1235
2919
3451
4489
5037
1269
2685
3465
4277
5019
1257
2569
3607
4179
5180
1315
2949
3523
4576
5405
1032
1771
2568
3567
4333
1279
1805
2629
3410
4457
1333
1906
2912
3678
4489
1106
1883
2930
3720
4501
1327
2219
2799
3590
4374
838
2422
3189
3933
4711
811
2361
3140
3995
4715
819
2380
3431
4284
4900
795
2420
3298
4170
5143
836
2573
3296
4077
4910
796
2411
3129
3871
4629
778
2415
3280
3997
4806
828
2439
3203
4024
4816
816
2385
3156
4019
4723
833
2513
3289
4092
4924
815
1538
2351
3116
3882
839
1636
2356
3143
3973
971
1680
2482
3310
4061
852
1640
2449
3385
4039
940
1738
2544
3369


2529
3375
4030
4916
906
2512
3305
4097
4943
917
2512
3335
4125
4951
890
2522
3252
3965
4712
871
2543
3312
4086
4907
890
2525
3331
4144
4901
875
2439
3214
4025
4800
924
2564
3376
4184
5018
893
1625
2415
3155
4058
898
1710
2506
3233
4048
891
1742
2493
3311
4098
976
1731
2534
3290
4143
992
1796
2605
3528
4208
1503
3086
3811
4574
5353
1439
3024
3858
4611
5379
1559
3132
3966
4731
5507
1506
3133
3956
4752
5561
1516
3150
3946
4769
5482
1485
3029
3790
4523
5282
1506
3126
3935
4710
5465
1575
3138
3909
4686
5437
1486
3000
3848
4625
5486
1612
3182
4055
4868
5641
1471
2210
3002
3771
4619
1501
2287
3104
3850
4759
1545
2328
3101
4004
4809
1547
2398
3172
3890
4709
1646
2498
3208
4085
4828
1377
2903
3614
4433
5146
1313
2859
3703
4480
5320
1387
3007
3750
4586
5351
1337
3026
3792
4706
5401
1450
3000
3794
4662
5326
1386
2906
3629
4484
5243
1419
2982
3756
4644
5299
1443
2975
3860
4540
5312
1313
2939
3758
4429
5320
1466
3133
3853
4651
5594
1340
2059
2857
3715
4463
1343
2238
2939
3722
4614
1396
2274
3050
37

...

2729
3576
4300
5096
1159
2739
3517
4311
5082
1073
2617
3448
4257
5064
1181
2794
3647
4397
5257
1121
1873
2619
3335
4145
1105
1874
2575
3446
4270
1150
1966
2767
3568
4324
1152
1987
2763
3541
4334
1220
2033
2847
3684
4439
1038
2600
3351
4111
4863
972
2538
3356
4145
4927
955
2545
3389
4238
4975
993
2577
3461
4220
5050
1016
2639
3461
4216
5015
976
2588
3328
4056
4826
972
2610
3435
4195
4985
978
2635
3402
4216
4978
988
2506
3337
4092
4913
1024
2661
3429
4206
5041
972
1662
2540
3263
4064
1010
1768
2521
3361
4147
1027
1861
2623
3366
4151
1041
1847
2636
3434
4220
1048
1929
2753
3540
4356
977
2529
3287
4105
4803
903
2530
3286
4144
4865
896
2495
3332
4163
4975
989
2583
3398
4227
5054
965
2649
3402
4161
5011
979
2580
3328
4003
4771
911
2614
3375
4158
4989
983
2578
3400
4220
4980
927
2498
3284
4092
4846
976
2632
3429
4233
5061
972
1679
2477
3206
4002
956
1774
2521
3301
4147
1031
1807
2573
3366
4151
1045
1775
2592
3440
4223
1051
1867
2680
3479
4298
954
2512
3278
3926
4796
887
2355
3253
3984
4844
88

955
2513
3278
4030
4796
895
2450
3254
4071
4855
886
2487
3311
4141
4965
908
2576
3394
4079
5019
939
2503
3395
4138
4937
892
2504
3244
3996
4767
908
2530
3356
4150
4913
943
2569
3319
4141
4897
918
2421
3277
4086
4832
969
2594
3406
4209
5041
914
1670
2470
3189
3985
947
1702
2508
3265
4140
887
1772
2570
3360
4134
979
1753
2555
3291
4136
974
1830
2666
3459
4253
1034
2597
3336
4036
4861
965
2457
3346
4080
4922
935
2739
3532
4371
4986
913
2640
3441
4079
5206
1014
2503
3447
4216
4936
897
2516
3244
4037
4802
954
2540
3417
4180
4918
958
2634
3326
4151
4909
970
2433
3323
4153
4899
1024
2646
3505
4290
5122
957
1741
2522
3251
4064
991
1709
2569
3342
4162
891
1858
2612
3417
4219
987
1834
2636
3303
4147
986
1910
2729
3540
4352
910
2428
3217
4079
4669
759
2499
3144
4120
4775
756
2380
3194
4030
4819
961
2483
3265
4164
4912
894
2582
3310
4099
4979
943
2550
3294
3925
4690
782
2576
3230
4057
4956
862
2495
3373
4187
4949
850
2464
3188
3939
4773
899
2539
3344
4155
4926
863
1548
2381
3133
3927
865
1736
2481

4274
5062
1067
2677
3461
4277
5016
1040
2531
3372
4178
4948
1067
2723
3523
4356
5171
1029
1770
2567
3306
4117
1031
1805
2606
3384
4220
1076
1906
2670
3458
4250
1106
1868
2689
3484
4264
1102
1967
2769
3573
4371
1198
2639
3507
4261
5042
1126
2719
3506
4313
4976
992
2756
3557
4386
5195
1145
2814
3612
4434
5244
1180
2710
3496
4273
5170
1117
2744
3503
4234
4999
1130
2777
3614
4366
5112
1160
2788
3446
4382
5044
1143
2691
3511
4308
5085
1214
2738
3669
4463
5282
1150
1894
2697
3426
4212
1149
1835
2646
3399
4372
1234
2019
2798
3600
4405
1170
2011
2812
3604
4290
1267
1988
2930
3721
4390
889
2428
3156
4027
4711
763
2449
3147
4074
4715
761
2387
3194
4026
4830
905
2427
3305
4062
4880
832
2465
3246
4038
4929
889
2507
3237
3873
4634
782
2529
3230
4000
4903
808
2441
3311
4140
4893
784
2425
3122
3935
4722
835
2482
3337
4099
4924
863
1541
2318
3096
3881
857
1630
2477
3202
3978
856
1710
2434
3220
4058
970
1640
2450
3287
4140
973
1770
2547
3370
4188
1035
2516
3278
4061
4861
968
2487
3349
4090
4923
889
254

4574
1481
2307
3056
3937
4689
912
2427
3170
4082
4683
763
2502
3150
4126
4705
792
2387
3194
4027
4849
964
2427
3277
4176
4880
908
2590
3246
4038
4984
950
2556
3297
3874
4636
782
2583
3230
4003
4959
871
2442
3378
4190
4953
800
2470
3122
3974
4723
836
2483
3360
4101
4926
884
1548
2335
3145
3928
876
1743
2478
3219
3981
978
1723
2438
3248
4044
1019
1711
2450
3387
4194
1022
1785
2548
3370
4204
1035
2598
3346
4069
4861
969
2491
3350
4097
4923
943
2547
3386
4235
4993
935
2640
3448
4142
5034
1014
2562
3454
4216
4965
937
2535
3268
4045
4805
958
2565
3423
4186
4943
961
2634
3348
4172
4940
977
2457
3326
4154
4902
1024
2656
3508
4299
5125
967
1746
2528
3258
4064
999
1729
2572
3350
4165
924
1859
2619
3419
4220
998
1839
2640
3355
4168
1004
1918
2739
3541
4353
1016
2528
3286
4112
4821
950
2540
3331
3996
4879
881
2541
3350
4163
4975
994
2542
3416
4227
4965
965
2649
3420
4155
5016
979
2415
3148
4003
4771
929
2616
3375
4196
4989
975
2540
3400
4220
4980
927
2498
3243
4068
4846
955
2632
3429
4233
5061
972

1156
1958
2713
3505
4327
1148
1944
2750
3537
4318
1132
2029
2857
3636
4443
1038
2590
3351
4112
4870
972
2540
3356
4147
4927
948
2545
3389
4238
4975
994
2644
3453
4227
5047
1016
2651
3466
4216
5016
982
2588
3328
4057
4829
973
2616
3435
4193
4989
978
2589
3402
4222
4981
989
2505
3338
4092
4907
1011
2683
3511
4310
5129
972
1758
2540
3263
4064
1010
1775
2586
3362
4170
1034
1866
2634
3423
4229
1048
1847
2631
3442
4223
1052
1930
2750
3549
4356
1177
2751
3486
4243
5035
1113
2687
3531
4306
5081
1107
2739
3526
4371
5177
1140
2777
3596
4399
5206
1165
2826
3604
4379
5152
1147
2728
3481
4211
4970
1117
2769
3580
4353
5130
1147
2772
3553
4367
5114
1125
2668
3494
4289
5067
1187
2830
3654
4452
5260
1119
1876
2678
3410
4194
1143
1936
2559
3507
4331
1216
2004
2774
3564
4370
1200
1990
2800
3582
4397
1235
2070
2899
3687
4475
1163
2707
3472
4261
5042
1119
2697
3472
4313
5047
1084
2759
3539
4385
5195
1145
2793
3562
4368
5237
1175
2794
3586
4337
5160
1117
2715
3450
4234
4998
1130
2777
3590
4335
5112
1160
274

In [31]:
for i in range(100):
    print(accum_res[i])

3950
2020
7528
4541
6624
4099
5039
5571
8200
6201
7304
3094
2172
3246
3833
4087
4123
1859
4390
7464
7539
2624
2795
3664
2466
1284
2574
6548
2377
3975
4599
6676
3270
8911
2847
2713
2183
6062
1583
2238
1844
3432
4337
9912
6626
9881
2463
2015
4932
2431
3278
2340
3219
4740
2584
8168
8841
3622
5339
2679
4145
2339
6509
4931
2574
1795
2356
2947
3859
1849
9912
4466
7008
4076
5528
4300
2192
3204
2859
2139
4713
7801
2545
4435
2942
4108
2535
6429
2519
3414
3275
4884
2589
2186
2744
4456
8101
1691
5737
4239


In [32]:
y_pred = [(a/N_ESTIMATORS)*FROM_RANGE/TO_RANGE for a in accum_res]

mse_int = mean_squared_error(y_test.values[0:N_SAMPLES], y_pred)
rmse_int = mse_int**.5
print(mse_int)
print(rmse_int)

print(percent_dif(rmse,rmse_int))

0.32669758836429996
0.5715746568597141
0.0007615557578213211


In [33]:
# for sample in X_test.values:
#     for value in sample:
#         (att_float_hex,_) = float16_bin_IEEE754(value)
# #         print(float(value), end=' ')
#         print(att_float_hex, end=' ')
#     print('')

# FILE GEN

In [37]:
for i in range(len(bin_tree_rams)):
    tree_coeff_file_gen(bin_tree_rams[i], 'tree_ram_15dtp_'+str(i))

In [30]:
attribute_coeff_file_gen(X_test.values[0:N_SAMPLES], 'attr_test_rgs')

In [34]:
sample_mem_file_gen(bin_sample,'samples_mem_rgs_15dtp')

In [35]:
tree_mem_file_gen(bin_tree_rams,'dtps_mem_rgs_15dtp')

In [19]:
print(len(bin_tree_rams[0])+len(bin_tree_rams[1])+len(bin_tree_rams[2])+len(bin_tree_rams[3]))

79358


17201 19328 17473 15408 26242 16843 20536 55134 
16908 20288 17836 15367 27258 16603 20728 55197 
18013 20960 18053 15425 24936 16556 20669 55204 
16850 20896 17611 15074 24778 16921 20663 55207 
18163 19584 18152 15320 27046 16772 20647 55197 
17351 20032 17555 15403 26331 16719 20648 55198 
17824 19200 17849 15400 28627 16934 20658 55200 
17911 19072 18423 15467 26414 16819 20582 55178 
17157 21120 17477 15400 24818 16050 20664 55206 
18169 19456 18116 15301 25855 17069 20532 55131 
17699 20896 17597 15371 23000 16448 20548 55142 
16489 20960 17733 15589 25342 16805 20544 55141 
16793 19712 17860 15393 25781 16906 20708 55195 
17682 19328 18903 16287 25520 16896 20701 55166 
17786 19904 18097 15383 25218 17013 20693 55188 
17644 19840 18107 15361 26371 16859 20632 55162 
17444 18944 17742 15489 24608 16345 20507 55123 
16883 20544 18023 15429 24984 17011 20628 55166 
16975 20160 17569 15445 26598 16634 20549 55142 
16892 20576 17074 15435 25910 15906 20544 55144 
18394 18560 17922 15

In [109]:
float16_bin_IEEE754(X_test.values[999][7].item())

('0xD797', '1101011110010111')

In [1]:
def read_accelerator_result(filename):
    res_buffer = []
    with open(filename,'r') as f:
        for res in f:
            res_buffer.append(int(res))
    return res_buffer

In [29]:
accelerator_res = read_accelerator_result('accelerator_result.txt')
for i in range(len(accelerator_res)):
    print(i, accelerator_res[i])

0 3950
1 2020
2 7528
3 4541
4 6624
5 4099
6 5039
7 5571
8 8200
9 6201
10 7304
11 3094
12 2172
13 3246
14 3833
15 4087
16 4123
17 1859
18 4390
19 7464
20 7539
21 2624
22 2795
23 3664
24 2466
25 1284
26 2574
27 6548
28 2377
29 3975
30 4599
31 6676
32 3270
33 8911
34 2847
35 2713
36 2183
37 6062
38 1583
39 2238
40 1844
41 3432
42 4337
43 9912
44 6626
45 9881
46 2463
47 2015
48 4932
49 2431
50 3278
51 2340
52 3219
53 4740
54 2584
55 8168
56 8841
57 3622
58 5339
59 2679
60 4145
61 2339
62 6509
63 4931
64 2574
65 1795
66 2356
67 2947
68 3859
69 1849
70 9912
71 4466
72 7008
73 4076
74 5528
75 4300
76 2192
77 3204
78 2859
79 2139
80 4713
81 7801
82 2545
83 4435
84 2942
85 4108
86 2535
87 6429
88 2519
89 3414
90 3275
91 4884
92 2589
93 2186
94 2744
95 4456
96 8101
97 1691
98 5737
99 4239
100 5923
101 6560
102 4931
103 5423
104 2872
105 2605
106 3974
107 1302
108 3225
109 5287
110 5688
111 5188
112 4032
113 1641
114 3585
115 5192
116 2730
117 5029
118 4336
119 4220
120 1509
121 6096
122 3065
123

In [102]:
def compare_buffer(b1,b2):
    if len(b1) != len(b2):
        return False
    is_true = True
    false_count = 0
    for i in range(len(b1)):
        if b1[i] != b2[i]:
            print(i,b1[i],b2[i])
            false_count +=1
            is_true = False
    print(false_count)
    return is_true

def compare_buffer_bin(b1,b2):
    if len(b1) != len(b2):
        return False
    is_true = True
    false_count = 0
    for i in range(len(b1)):
        if b1[i][:30] != b2[i][:30] or  b1[i][31] != b2[i][31]:
            print(i,b1[i],b2[i])
            false_count +=1
            is_true = False
    print(false_count)
    return is_true

In [35]:
compare_buffer(accelerator_res, accum_res)

287 6664 6625


False

In [36]:
y_pred = [(a/N_ESTIMATORS)*FROM_RANGE/TO_RANGE for a in accelerator_res]

mse_int = mean_squared_error(y_test.values[0:N_SAMPLES], y_pred)
rmse_int = mse_int**.5
print(mse_int)
print(rmse_int)

print(percent_dif(rmse,rmse_int))

0.3267979061143
0.5716624057206315
0.0009150652065649377


In [ ]:
#=====================================================================

In [ ]:
#=====================================================================

In [ ]:
#=====================================================================

In [ ]:
#=====================================================================

In [ ]:
#=====================================================================

In [ ]:
#=====================================================================

In [ ]:
#=====================================================================

In [ ]:
#=====================================================================

In [ ]:
#=====================================================================

In [ ]:
#=====================================================================

In [ ]:
#=====================================================================

In [ ]:
#=====================================================================

In [ ]:
#=====================================================================

In [ ]:
#=====================================================================

# GLASS CLASSIFICATION

In [ ]:
# useful link
# https://datascience.stackexchange.com/questions/11171/decision-tree-how-to-understand-or-calculate-the-probability-confidence-of-pred
# http://rasbt.github.io/mlxtend/user_guide/classifier/EnsembleVoteClassifier/

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [7]:
data = pd.read_csv("glass.csv")

In [8]:
data.info()
data.duplicated().sum() # check duplicated data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214 entries, 0 to 213
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   RI      214 non-null    float64
 1   Na      214 non-null    float64
 2   Mg      214 non-null    float64
 3   Al      214 non-null    float64
 4   Si      214 non-null    float64
 5   K       214 non-null    float64
 6   Ca      214 non-null    float64
 7   Ba      214 non-null    float64
 8   Fe      214 non-null    float64
 9   Type    214 non-null    int64  
dtypes: float64(9), int64(1)
memory usage: 16.8 KB


1

In [9]:
data.head(5)

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.0,0.0,1
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.0,0.0,1
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.0,0.0,1
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.0,0.0,1
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.0,0.0,1


In [10]:
data.drop_duplicates(inplace=True)
data.duplicated().sum()

0

In [11]:
# Import train_test_split function
from sklearn.model_selection import train_test_split

X=data[['RI', 'Na', 'Mg', 'Al', 'Si', 'K', 'Ca', 'Ba', 'Fe']]  # Features
y=data['Type']  # Labels

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70% training and 30% test

In [12]:
X_train_file = 'X_train_clf.csv'
X_test_file  = 'X_test_clf.csv'
y_train_file = 'y_train_clf.csv'
y_test_file  = 'y_test_clf.csv'

In [13]:
# save dataframe to csv
# X_train.to_csv(X_train_file, sep='\t', encoding='utf-8', index=False)
# X_test.to_csv(X_test_file, sep='\t', encoding='utf-8', index=False)
# y_train.to_csv(y_train_file, sep='\t', encoding='utf-8', index=False)
# y_test.to_csv(y_test_file, sep='\t', encoding='utf-8', index=False)

# LOADING DATASET

In [6]:
# load csv to dataframe
import pandas as pd
X_train = pd.read_csv('X_train_clf.csv', sep='\t', encoding='utf-8')
X_test  = pd.read_csv('X_test_clf.csv', sep='\t', encoding='utf-8')
y_train = pd.read_csv('y_train_clf.csv', sep='\t', encoding='utf-8')
y_test  = pd.read_csv('y_test_clf.csv', sep='\t', encoding='utf-8')

In [7]:
from IPython.display import display, HTML
display(X_train[0:5])
display(X_test[0:5])
display(y_train[0:5])
display(y_test[0:5])
print(len(y_train), len(y_test))

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe
0,1.51909,13.89,3.53,1.32,71.81,0.51,8.78,0.11,0.00
1,1.51409,14.25,3.09,2.08,72.28,1.10,7.08,0.00,0.00
2,1.51789,13.19,3.90,1.30,72.33,0.55,8.44,0.00,0.28
3,1.51711,14.23,0.00,2.08,73.36,0.00,8.62,1.67,0.00
4,1.51609,15.01,0.00,2.51,73.05,0.05,8.83,0.53,0.00


,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe
0,1.51755,12.71,3.42,1.20,73.20,0.59,8.64,0.00,0.00
1,1.51969,12.64,0.00,1.65,73.75,0.38,11.53,0.00,0.00
2,1.51556,13.87,0.00,2.54,73.23,0.14,9.41,0.81,0.01
3,1.51602,14.85,0.00,2.38,73.28,0.00,8.76,0.64,0.09
4,1.51683,14.56,0.00,1.98,73.29,0.00,8.52,1.57,0.07


,Type
0,1
1,2
2,2
3,7
4,7


,Type
0,1
1,5
2,7
3,7
4,7


149 64


In [8]:
from sklearn.preprocessing import MinMaxScaler

# # scaler = MinMaxScaler(feature_range=(0, 2**16-1))
# scaler = MinMaxScaler(feature_range=(0, 1000))
# X_train_norm = scaler.fit_transform(X_train)
# X_test_norm = scaler.fit_transform(X_test)

# X_train_norm = pd.DataFrame(X_train_norm, columns=['RI', 'Na', 'Mg', 'Al', 'Si', 'K', 'Ca', 'Ba', 'Fe'])
# X_test_norm = pd.DataFrame(X_test_norm, columns=['RI', 'Na', 'Mg', 'Al', 'Si', 'K', 'Ca', 'Ba', 'Fe'])

In [9]:
X_train_norm = X_train.copy()
X_test_norm = X_test.copy()

In [10]:
# X_train_norm['RI'] = X_train_norm['RI']*1000
# X_train_norm['RI'] = (X_train_norm['RI']-X_train_norm['RI'].mean())/X_train_norm['RI'].std()
scaler = MinMaxScaler(feature_range=(0, 2047))
X_train_norm['RI'] = scaler.fit_transform(X_train_norm[['RI']])

X_train_norm['Na'] = X_train_norm['Na']*100
X_train_norm['Mg'] = X_train_norm['Mg']*100
X_train_norm['Al'] = X_train_norm['Al']*100
X_train_norm['Si'] = X_train_norm['Si']*100
X_train_norm['K' ] = X_train_norm['K' ]*100
X_train_norm['Ca'] = X_train_norm['Ca']*100
X_train_norm['Ba'] = X_train_norm['Ba']*100
X_train_norm['Fe'] = X_train_norm['Fe']*100


# X_test_norm['RI'] = X_test_norm['RI']*1000
# X_test_norm['RI'] = (X_test_norm['RI']-X_test_norm['RI'].mean())/X_test_norm['RI'].std()
X_test_norm['RI'] = scaler.transform(X_test_norm[['RI']])

X_test_norm['Na'] = X_test_norm['Na']*100
X_test_norm['Mg'] = X_test_norm['Mg']*100
X_test_norm['Al'] = X_test_norm['Al']*100
X_test_norm['Si'] = X_test_norm['Si']*100
X_test_norm['K' ] = X_test_norm['K' ]*100
X_test_norm['Ca'] = X_test_norm['Ca']*100
X_test_norm['Ba'] = X_test_norm['Ba']*100
X_test_norm['Fe'] = X_test_norm['Fe']*100

In [11]:
# X_train_norm=(X_train-X_train.mean())/X_train.std()
# X_test_norm=(X_test-X_test.mean())/X_test.std()

In [12]:
# for c in X_train_norm.columns:
#     X_train_norm[c] = X_train_norm[c].astype('float').astype('int')
# for c in X_test_norm.columns:
#     X_test_norm[c] = X_test_norm[c].astype('float').astype('int')

In [13]:
display(X_train_norm[0:5])
display(X_test_norm[0:5])
display(y_train[0:5])
display(y_test[0:5])
print(len(y_train), len(y_test))

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe
0,584.434762,1389.0,353.0,132.0,7181.0,51.0,878.0,11.0,0.0
1,91.656716,1425.0,309.0,208.0,7228.0,110.0,708.0,0.0,0.0
2,466.168031,1319.0,390.0,130.0,7233.0,55.0,844.0,0.0,28.0
3,389.294656,1423.0,0.0,208.0,7336.0,0.0,862.0,167.0,0.0
4,288.767935,1501.0,0.0,251.0,7305.0,5.0,883.0,53.0,0.0


,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe
0,432.659124,1271.0,342.0,120.0,7320.0,59.0,864.0,0.0,0.0
1,643.568127,1264.0,0.0,165.0,7375.0,38.0,1153.0,0.0,0.0
2,236.533462,1387.0,0.0,254.0,7323.0,14.0,941.0,81.0,1.0
3,281.869042,1485.0,0.0,238.0,7328.0,0.0,876.0,64.0,9.0
4,361.699085,1456.0,0.0,198.0,7329.0,0.0,852.0,157.0,7.0


,Type
0,1
1,2
2,2
3,7
4,7


,Type
0,1
1,5
2,7
3,7
4,7


149 64


In [14]:
# from sklearn.model_selection import RandomizedSearchCV
# import numpy as np
# # Number of trees in random forest
# n_estimators = [int(x) for x in np.linspace(start = 100, stop = 5000, num = 20)]
# # Number of features to consider at every split
# # max_features = ['auto', 'sqrt']
# # Maximum number of levels in tree
# max_depth = [int(x) for x in np.linspace(4, 10,num=7)]
# max_depth.append(None)
# # Minimum number of samples required to split a node
# # min_samples_split = [2, 5, 10]
# # Minimum number of samples required at each leaf node
# # min_samples_leaf = [1, 2, 4]
# # Method of selecting samples for training each tree
# bootstrap = [True, False]
# # Create the random grid
# random_grid = {'n_estimators': n_estimators,
# #                'max_features': max_features,
#                'max_depth': max_depth,
# #                'min_samples_split': min_samples_split,
# #                'min_samples_leaf': min_samples_leaf,
#                'bootstrap': bootstrap}
# print(random_grid)

In [15]:
# # Use the random grid to search for best hyperparameters
# # First create the base model to tune
# rf = RandomForestClassifier()
# # Random search of parameters, using 3 fold cross validation, 
# # search across 100 different combinations, and use all available cores
# rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 200, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# # Fit the random search model
# rf_random.fit(X_train_norm, y_train.values.ravel())

In [16]:
# rf_random.best_params_

In [17]:
# #Import Random Forest Model
# from sklearn.ensemble import RandomForestClassifier

# # Create a Gaussian Classifier
# clf=RandomForestClassifier(n_estimators=1000, max_depth=9, bootstrap=True)
# # clf=RandomForestClassifier(n_estimators=100, bootstrap=True)
# # clf=RandomForestClassifier()
# # clf=RandomForestClassifier(max_depth=4)

# #Train the model using the training sets y_pred=clf.predict(X_test)
# clf.fit(X_train_norm,y_train.values.ravel())
# # clf.fit(X_train,y_train.values.ravel())

In [18]:
# import joblib

# # save
# joblib.dump(clf, "glass_clf.joblib")

# LOADING MODEL

In [19]:
import joblib
clf = joblib.load("./glass_clf.joblib")

In [20]:
print(clf.classes_)

[1 2 3 5 6 7]


In [21]:
# y_pred=clf.predict(X_test)
y_pred=clf.predict(X_test_norm)

In [22]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.84375


In [23]:
import numpy as np
def scikit_decision_path(sample,rf): 
    base_idx = 5
    trees = rf.estimators_
    predicts = []
#     leaf_indices = []
    for s_idx in range(len(sample.values)):
        s = sample.values[s_idx]
        results = []
#         print('Sample:',s_idx,'=======================================================================================')
        for t_idx in range(len(trees)):
#             print('Tree:',t_idx,'=======================================================================================')
            tree = trees[t_idx].tree_
            i = 0
            while(tree.children_left[i] != tree.children_right[i]):
#                 print(base_idx+i, tree.feature[i].item(), s[tree.feature[i].item()], tree.threshold[i].item())
                if(s[tree.feature[i].item()] <= tree.threshold[i].item()):
                    i = tree.children_left[i].item()
                else:
                    i = tree.children_right[i].item()
#             print(i, tree.feature[i].item(), s[tree.feature[i].item()], tree.threshold[i].item(), end=' ')
#             leaf_indices.append(i)
#             print('leaf:', tree.value[i][0], )
            results.append(rf.classes_[np.argmax(tree.value[i]).item()])
            base_idx += tree.node_count
        predicts.append(max(set(results), key = results.count))
    return predicts
#     return leaf_indices

In [24]:
predicts_sw = scikit_decision_path(X_test_norm,clf)
# leaves = scikit_decision_path(X_test_norm[21:22],clf)
for i in range(len(predicts_sw)):
    print(i,predicts_sw[i], y_pred[i])

0 1 1
1 5 5
2 7 7
3 7 7
4 7 7
5 1 1
6 1 1
7 2 2
8 3 3
9 2 2
10 7 7
11 3 3
12 1 1
13 2 2
14 7 7
15 3 3
16 2 2
17 6 6
18 1 1
19 1 1
20 1 1
21 2 2
22 7 7
23 2 2
24 2 2
25 1 1
26 5 5
27 2 2
28 1 2
29 2 2
30 7 7
31 2 2
32 6 6
33 2 2
34 2 2
35 1 1
36 1 1
37 5 5
38 1 1
39 1 1
40 1 1
41 1 1
42 2 2
43 1 1
44 7 7
45 2 2
46 1 1
47 2 2
48 2 2
49 2 2
50 7 7
51 7 7
52 1 1
53 5 5
54 7 7
55 7 7
56 7 7
57 1 1
58 7 7
59 5 5
60 7 7
61 2 2
62 7 7
63 1 1


In [25]:
print("Accuracy:",metrics.accuracy_score(y_test, predicts_sw))

Accuracy: 0.828125


In [26]:
def print_tree(tree):
    accum_node = 0
    for i in range(len(tree.estimators_)):
        print("Tree",i,'-----------------------------------------------------------------------------------')
        print("Number of nodes: ", tree.estimators_[i].tree_.node_count) #Number of node
#         print("Number of nodes: ", tree.estimators_[i].tree_.value) #Number of node
#         print("array of left children: ", tree.estimators_[i].tree_.children_left) #array of left children
#         print("array of right children: ", tree.estimators_[i].tree_.children_right) #array of right children
#         print("array of nodes splitting feature: ", tree.estimators_[i].tree_.feature) #array of nodes splitting feature
#         print("array of nodes splitting points: ", tree.estimators_[i].tree_.threshold) #array of nodes splitting points
        accum_node += tree.estimators_[i].tree_.node_count
    print('Total number of nodes:', accum_node)

In [27]:
print_tree(clf)

Tree 0 -----------------------------------------------------------------------------------
Number of nodes:  67
Tree 1 -----------------------------------------------------------------------------------
Number of nodes:  55
Tree 2 -----------------------------------------------------------------------------------
Number of nodes:  75
Tree 3 -----------------------------------------------------------------------------------
Number of nodes:  61
Tree 4 -----------------------------------------------------------------------------------
Number of nodes:  71
Tree 5 -----------------------------------------------------------------------------------
Number of nodes:  69
Tree 6 -----------------------------------------------------------------------------------
Number of nodes:  73
Tree 7 -----------------------------------------------------------------------------------
Number of nodes:  63
Tree 8 -----------------------------------------------------------------------------------
Number of nod

In [28]:
import time

exec_time_list = []
accum = 0
for i in range(20):
    start_time = time.time()
    clf.predict(X_test_norm)
    end_time = time.time()
    exec_time = end_time - start_time
    print(exec_time)
    accum += exec_time
    exec_time_list.append(exec_time)
print('mean:',accum/len(exec_time_list))

0.33516645431518555
0.25925517082214355
0.37200117111206055
0.3776113986968994
0.44870448112487793
0.40215611457824707
0.334871768951416
0.27629613876342773
0.257279634475708
0.2682812213897705
0.2520756721496582
0.34060144424438477
0.3186366558074951
0.351123571395874
0.2723681926727295
0.3070187568664551
0.2723269462585449
0.26727867126464844
0.24529552459716797
0.2682833671569824
mean: 0.3113316178321838


In [29]:
float16_bin_IEEE754(1517.55)
# float16_bin_IEEE754(0.029999999329447746)

('0x65ED', '0110010111101101')

# BIN RAM GEN

In [30]:
BRAM_DWIDTH      = 32
BRAM_AWIDTH      = 14

TREE_DEPTH       = 10 # number of levels
LEAF_IDX         = 0
THSH_IDX         = 1
THSH_WIDTH       = 16
ATTR_IDX         = 17
ATTR_WIDTH       = 5
RCHLD_IDX        = 22
RCHLD_WIDTH      = BRAM_DWIDTH - ATTR_WIDTH - THSH_WIDTH - 1
LAST_IDX         = 1
RES_IDX          = 2
RES_WIDTH        = 16
TREE_IDX         = 18
TREE_WIDTH       = BRAM_DWIDTH - RES_WIDTH - 2

LAST_AB_IDX      = 0
AB_TREE_IDX      = 1
AB_TREE_WIDTH    = BRAM_AWIDTH

IS_INT = False
IS_CLF = True

In [31]:
N_ESTIMATORS = 1000
N_DTPS = 5
PIPELINE_STAGES = 5

bin_tree_rams = [] # list of Tree RAMs
dtp_trees = list(split(clf.estimators_, N_DTPS)) # list of subset of tree (Each subset belongs to each DTP)
for subset in dtp_trees:
    bin_bram = []
    tree_bram_gen(dtree_list=subset, no_parts=PIPELINE_STAGES, tree_bram=bin_bram)
    bin_tree_rams.append(bin_bram)

In [32]:
for bin_tree_ram in bin_tree_rams:
    print(len(bin_tree_ram))

12485
12473
12723
12457
12405


# DEC RAM GEN

In [33]:
dec_tree_rams = []
for bin_tree_ram in bin_tree_rams:
    dec_tree_ram = []
    rambin2ramdec(bin_tree_ram, PIPELINE_STAGES, dec_tree_ram)
    dec_tree_rams.append(dec_tree_ram)

In [34]:
# for node in dec_tree_rams[0][0:100]:
#     print(node)

# BIN SAMPLE GEN

In [35]:
bin_sample = sample_dec2bin(X_test_norm) # list of samples in binary

In [36]:
# for i in range(len(bin_sample)):
#     print(i,end=' ')
#     for a in bin_sample[i]:
#         print(a, end=' ')
#     print('')

# PREDICTING

In [37]:
def decision_path(sample, ramdec, start_idx, results):
#     results = [0]*len(clf.classes_)
    idx = start_idx
    while True:
#         print(idx, end =' ') # for decision path
        if ramdec[idx]['is_leaf'] == True:
#             results.append(ramdec[idx]['result'])
            results[ramdec[idx]['result']] += 1
#             print('result:', ramdec[idx]['result'])
            if ramdec[idx]['is_last'] ==  True: 
#                 print(idx)
                break
            idx += ramdec[idx]['next_tree']
        else:
#             print(ramdec[idx]['attribute'],bin_float16_IEEE754(sample[ramdec[idx]['attribute']]), ramdec[idx]['threshold'])
            dec_attr = int(sample[ramdec[idx]['attribute']],2) if IS_INT else bin_float16_IEEE754(sample[ramdec[idx]['attribute']])
            
            if dec_attr <= ramdec[idx]['threshold']:
                idx += 1
            else:
                idx += ramdec[idx]['right_child'] + 1
#     return results

def decision_path_bram(sample_list, ramdec_list):
    predicts = [0]*len(clf.classes_)
    predicts = []
    for s_idx in range(len(sample_list)):
        print('Sample',s_idx,'****************************************************************************') # for decision path
        results = [0]*len(clf.classes_)
        for d_idx in range(len(ramdec_list)):
#             if d_idx!=0:
#                 continue
#             print('DTP',d_idx,'****************************************************************************')
            for node_idx in range(PIPELINE_STAGES):
#                 print('Stage', node_idx, '------------------------------------------') # for decision path
                decision_path(sample_list[s_idx], ramdec_list[d_idx], int(ramdec_list[d_idx][node_idx]['absolute_addr']), results)
                if ramdec_list[d_idx][node_idx]['is_last_absolute_addr'] == True:
                    break
        
#         for i in range(len(predicts)):
#             predicts[i] += results[i]
        print(results)
        predicts.append(results.index(max(results)))
    return predicts

predicts_hw = decision_path_bram(bin_sample, dec_tree_rams)

Sample 0 ****************************************************************************
[948, 25, 26, 0, 0, 1]
Sample 1 ****************************************************************************
[33, 133, 6, 774, 43, 11]
Sample 2 ****************************************************************************
[54, 158, 30, 138, 102, 518]
Sample 3 ****************************************************************************
[2, 11, 1, 4, 4, 978]
Sample 4 ****************************************************************************
[3, 15, 3, 2, 6, 971]
Sample 5 ****************************************************************************
[458, 354, 56, 26, 45, 61]
Sample 6 ****************************************************************************
[851, 131, 18, 0, 0, 0]
Sample 7 ****************************************************************************
[136, 712, 111, 28, 4, 9]
Sample 8 ****************************************************************************
[275, 41, 532, 0, 105, 47]


In [38]:
predicts_hw = [clf.classes_[p] for p in predicts_hw]

In [39]:
predicts_hw

[1,
 5,
 7,
 7,
 7,
 1,
 1,
 2,
 3,
 2,
 7,
 3,
 1,
 2,
 7,
 3,
 2,
 6,
 1,
 1,
 1,
 2,
 7,
 2,
 2,
 1,
 5,
 2,
 1,
 2,
 7,
 2,
 6,
 2,
 2,
 1,
 1,
 5,
 1,
 1,
 1,
 1,
 2,
 1,
 7,
 2,
 1,
 2,
 2,
 2,
 7,
 7,
 1,
 5,
 7,
 7,
 7,
 1,
 7,
 5,
 7,
 2,
 7,
 1]

In [40]:
# for i in range(len(predicts_hw)):
#     print(i,predicts_hw[i],predicts_sw[i])
for i in range(len(predicts_hw)):
    print(i,predicts_hw[i],y_test.values[i])

0 1 [1]
1 5 [5]
2 7 [7]
3 7 [7]
4 7 [7]
5 1 [1]
6 1 [1]
7 2 [2]
8 3 [1]
9 2 [7]
10 7 [7]
11 3 [1]
12 1 [1]
13 2 [2]
14 7 [7]
15 3 [3]
16 2 [6]
17 6 [6]
18 1 [2]
19 1 [1]
20 1 [1]
21 2 [2]
22 7 [7]
23 2 [2]
24 2 [2]
25 1 [1]
26 5 [5]
27 2 [2]
28 1 [2]
29 2 [2]
30 7 [7]
31 2 [2]
32 6 [6]
33 2 [2]
34 2 [2]
35 1 [3]
36 1 [1]
37 5 [5]
38 1 [1]
39 1 [1]
40 1 [1]
41 1 [1]
42 2 [2]
43 1 [1]
44 7 [7]
45 2 [2]
46 1 [1]
47 2 [2]
48 2 [7]
49 2 [2]
50 7 [7]
51 7 [7]
52 1 [1]
53 5 [2]
54 7 [7]
55 7 [7]
56 7 [5]
57 1 [1]
58 7 [7]
59 5 [5]
60 7 [7]
61 2 [6]
62 7 [7]
63 1 [1]


In [41]:
# print("Accuracy:",metrics.accuracy_score(y_test[0:1], predicts))
print("Accuracy:",metrics.accuracy_score(predicts_sw, predicts_hw))

Accuracy: 1.0


# FILES GEN

In [42]:
for i in range(len(bin_tree_rams)):
    tree_coeff_file_gen(bin_tree_rams[i], 'clf_ram_5dtp_'+str(i))

In [47]:
attribute_coeff_file_gen(X_test_norm.values, 'attr_test_clf')

In [43]:
sample_mem_file_gen(bin_sample,'clf_samples_mem')

In [44]:
tree_mem_file_gen(bin_tree_rams,'clf_dtps_mem_5dtp')

In [6]:
# Import packages-------------------------------------------------
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics


# load CSV to dataframe-------------------------------------------------
data = pd.read_csv("glass.csv")
data.drop_duplicates(inplace=True) # remove duplicated data

X=data[['RI', 'Na', 'Mg', 'Al', 'Si', 'K', 'Ca', 'Ba', 'Fe']]  # Features
y=data['Type']  # Labels

# Split dataset into training set and test set------------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70% training and 30% test


# Scale data-------------------------------------------------
X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()

scaler = MinMaxScaler(feature_range=(0, 2047))
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train_scaled), columns=X_train_scaled.columns)
# X_test_scaled  = pd.DataFrame(scaler.fit_transform(X_test_scaled), columns=X_test_scaled.columns)
X_test_scaled  = pd.DataFrame(scaler.transform(X_test_scaled), columns=X_test_scaled.columns)


# Train Random Forest-------------------------------------------------
# n_estimators=100, max_depth=9
clf = RandomForestClassifier()
clf_scaled = RandomForestClassifier()

clf.fit(X_train,y_train.values.ravel())
clf_scaled.fit(X_train_scaled,y_train.values.ravel())

# Predict and accuracy
y_pred = clf.predict(X_test)
y_pred_scaled = clf_scaled.predict(X_test_scaled)
print("Accuracy of non-scaled data:",metrics.accuracy_score(y_test, y_pred))
print("Accuracy of scaled data:",metrics.accuracy_score(y_test, y_pred_scaled))

Accuracy of non-scaled data: 0.828125
Accuracy of scaled data: 0.828125


In [22]:
#!/usr/bin/env python3

# Import packages-------------------------------------------------
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from IPython.display import display, HTML

def fnotScale ():
    # load CSV to dataframe-------------------------------------------------
    data = pd.read_csv("glass.csv")
    data.drop_duplicates(inplace=True) # remove duplicated data

    X=data[['RI', 'Na', 'Mg', 'Al', 'Si', 'K', 'Ca', 'Ba', 'Fe']]  # Features
    y=data['Type']  # Labels

    # Split dataset into training set and test set------------------------------------------------
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70% training and 30% test

    # Train Random Forest-------------------------------------------------
    # n_estimators=100, max_depth=9
    clf = RandomForestClassifier(n_estimators=1000, max_depth=9, bootstrap=True)
    clf.fit(X_train,y_train.values.ravel())

    # Predict and accuracy
    y_pred = clf.predict(X_test)
    print("Accuracy of non-scaled data:",metrics.accuracy_score(y_test, y_pred))

def fScale():
    # load CSV to dataframe-------------------------------------------------
    data = pd.read_csv("glass.csv")
    data.drop_duplicates(inplace=True) # remove duplicated data

    X=data[['RI', 'Na', 'Mg', 'Al', 'Si', 'K', 'Ca', 'Ba', 'Fe']]  # Features
    y=data['Type']  # Labels

    # Split dataset into training set and test set------------------------------------------------
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70% training and 30% test

    # Scale data-------------------------------------------------
    X_train_scaled = X_train.copy()
    X_test_scaled = X_test.copy()

#     scaler = MinMaxScaler(feature_range=(0, 2047))
    scaler = MinMaxScaler()
    X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train_scaled), columns=X_train_scaled.columns)
    X_test_scaled  = pd.DataFrame(scaler.transform(X_test_scaled), columns=X_test_scaled.columns)
    
    display(X_train_scaled[0:5])
    display(X_test_scaled[0:5])

    # Train Random Forest-------------------------------------------------
    # n_estimators=100, max_depth=9
    clf_scaled = RandomForestClassifier(n_estimators=1000, max_depth=9, bootstrap=True)
    clf_scaled.fit(X_train_scaled,y_train.values.ravel())

    # Predict and accuracy
    y_pred_scaled = clf_scaled.predict(X_test_scaled)
    print("Accuracy of scaled data:",metrics.accuracy_score(y_test, y_pred_scaled))

if __name__ == "__main__":
    fnotScale()
    fScale()

Accuracy of non-scaled data: 0.6875


,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe
0,0.183024,0.673913,0.000000,0.744548,0.755274,0.012882,0.338290,0.193651,0.098039
1,0.238285,0.496047,0.896985,0.339564,0.609705,0.090177,0.279740,0.000000,0.196078
2,0.233864,0.446640,0.798995,0.292835,0.666667,0.093398,0.314126,0.000000,0.470588
3,0.275420,0.525692,0.919598,0.280374,0.628692,0.091787,0.263941,0.000000,0.215686
4,0.316534,0.468379,0.997487,0.280374,0.554852,0.096618,0.278810,0.000000,0.000000


,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe
0,0.217507,0.673913,0.000000,0.806854,0.590717,0.012882,0.348513,0.336508,0.000000
1,0.354996,0.575099,0.889447,0.143302,0.599156,0.025765,0.321561,0.047619,0.470588
2,0.213086,0.622530,0.447236,0.467290,0.694093,0.000000,0.301115,0.241270,0.000000
3,0.460212,0.543478,0.939698,0.190031,0.464135,0.028986,0.388476,0.000000,0.137255
4,0.318744,0.664032,0.550251,0.426791,0.603376,0.000000,0.361524,0.000000,0.000000


Accuracy of scaled data: 0.78125


# LAND COVER

In [6]:
def print_tree(tree):
    accum_node = 0
    for i in range(len(tree.estimators_)):
        print("Tree",i,'-----------------------------------------------------------------------------------')
        print("Number of nodes: ", tree.estimators_[i].tree_.node_count) #Number of node
#         print("Number of nodes: ", tree.estimators_[i].tree_.value) #Number of node
#         print("array of left children: ", tree.estimators_[i].tree_.children_left) #array of left children
#         print("array of right children: ", tree.estimators_[i].tree_.children_right) #array of right children
#         print("array of nodes splitting feature: ", tree.estimators_[i].tree_.feature) #array of nodes splitting feature
#         print("array of nodes splitting points: ", tree.estimators_[i].tree_.threshold) #array of nodes splitting points
        accum_node += tree.estimators_[i].tree_.node_count
    print('Total number of nodes:', accum_node)

In [7]:
def decision_path(sample, ramdec, start_idx, results):
#     results = [0]*len(clf.classes_)
    idx = start_idx
    while True:
        print(idx, end =' ') # for decision path
        if ramdec[idx]['is_leaf'] == True:
            results[ramdec[idx]['result']] += 1
            print('idx: ',idx,',result:', ramdec[idx]['result'])
            if ramdec[idx]['is_last'] ==  True: 
                print(idx)
                break
            idx += ramdec[idx]['next_tree']
        else:
            if IS_INT:
                print(ramdec[idx]['attribute'],int(sample[ramdec[idx]['attribute']],2), ramdec[idx]['threshold'])
            else:
                print(ramdec[idx]['attribute'],bin_float16_IEEE754(sample[ramdec[idx]['attribute']]), ramdec[idx]['threshold'])
            
            dec_attr = int(sample[ramdec[idx]['attribute']],2) if IS_INT else bin_float16_IEEE754(sample[ramdec[idx]['attribute']])
            
            if dec_attr <= ramdec[idx]['threshold']:
                idx += 1
            else:
                idx += ramdec[idx]['right_child'] + 1

def decision_path_bram(sample_list, ramdec_list):
    predicts = []
    for s_idx in range(len(sample_list)):
        print('Sample',s_idx,'****************************************************************************') # for decision path
        results = [0]*len(clf.classes_)
        for d_idx in range(len(ramdec_list)):
#             if d_idx!=0:
#                 continue
            print('DTP',d_idx,'****************************************************************************')
            for node_idx in range(PIPELINE_STAGES):
                print('Stage', node_idx, '------------------------------------------') # for decision path
                decision_path(sample_list[s_idx], ramdec_list[d_idx], int(ramdec_list[d_idx][node_idx]['absolute_addr']), results)
                if ramdec_list[d_idx][node_idx]['is_last_absolute_addr'] == True:
                    break
        predicts.append(results)
        
#         for i in range(len(predicts)):
#             predicts[i] += results[i]
        print(results)
#         predicts.append(results.index(max(results)))
    return predicts

In [8]:
import numpy as np
def scikit_decision_path(sample,rf): 
    base_idx = 5
    trees = rf.estimators_
    predicts = []
#     leaf_indices = []
    for s_idx in range(len(sample.values)):
        s = sample.values[s_idx]
        results = []
#         print('Sample:',s_idx,'=======================================================================================')
        for t_idx in range(len(trees)):
#             print('Tree:',t_idx,'=======================================================================================')
            tree = trees[t_idx].tree_
            i = 0
            while(tree.children_left[i] != tree.children_right[i]):
#                 print(base_idx+i, tree.feature[i].item(), s[tree.feature[i].item()], tree.threshold[i].item())
                if(s[tree.feature[i].item()] <= tree.threshold[i].item()):
                    i = tree.children_left[i].item()
                else:
                    i = tree.children_right[i].item()
#             print(i, tree.feature[i].item(), s[tree.feature[i].item()], tree.threshold[i].item(), end=' ')
#             leaf_indices.append(i)
#             print('leaf:', tree.value[i][0], )
            results.append(rf.classes_[np.argmax(tree.value[i]).item()])
            base_idx += tree.node_count
        predicts.append(max(set(results), key = results.count))
    return predicts
#     return leaf_indices

## Prepare data 

In [9]:
# load csv to dataframe
import pandas as pd
X_train = pd.read_csv('../biomass/land_X_train.csv', sep='\t', encoding='utf-8').astype(float)
X_test  = pd.read_csv('../biomass/land_X_test.csv', sep='\t', encoding='utf-8').astype(float)
y_train = pd.read_csv('../biomass/land_y_train.csv', sep='\t', encoding='utf-8')
y_test  = pd.read_csv('../biomass/land_y_test.csv', sep='\t', encoding='utf-8')

In [10]:
display(X_train[0:5])
display(X_test[0:5])
display(y_train[0:5])
display(y_test[0:5])
print(len(y_test), len(y_train))

,Band_1,Band_2,Band_3,Band_4,Band_5,Band_6,Band_7
0,975.0,1383.0,1870.0,2841.0,3475.0,2919.0,3171.0
1,1393.0,1883.0,2328.0,3410.0,3848.0,3154.0,3313.0
2,306.0,584.0,355.0,4152.0,1867.0,727.0,2537.0
3,285.0,428.0,257.0,3584.0,1463.0,541.0,2487.0
4,397.0,643.0,505.0,3506.0,2211.0,962.0,2637.0


,Band_1,Band_2,Band_3,Band_4,Band_5,Band_6,Band_7
0,693.0,895.0,1098.0,2121.0,3224.0,2682.0,3171.0
1,522.0,674.0,748.0,1715.0,909.0,488.0,2436.0
2,306.0,517.0,336.0,3811.0,1817.0,753.0,2537.0
3,306.0,473.0,277.0,3213.0,1364.0,488.0,2537.0
4,241.0,428.0,257.0,3299.0,1564.0,594.0,2436.0


,0
0,5
1,4
2,3
3,1
4,3


,0
0,4
1,2
2,1
3,1
4,1


144 574


In [11]:
X_train.agg(['min', 'max'])

,Band_1,Band_2,Band_3,Band_4,Band_5,Band_6,Band_7
min,240.0,315.0,198.0,1250.0,681.0,331.0,2386.0
max,1642.0,2185.0,2687.0,4789.0,4548.0,3705.0,3546.0


In [12]:
X_test.agg(['min', 'max'])

,Band_1,Band_2,Band_3,Band_4,Band_5,Band_6,Band_7
min,241.0,338.0,178.0,1164.0,455.0,304.0,2386.0
max,1521.0,2014.0,2499.0,4734.0,3851.0,3300.0,3546.0


# RANDOM FOREST

In [13]:
# #Import Random Forest Model
# from sklearn.ensemble import RandomForestClassifier

# clf=RandomForestClassifier(n_estimators=1500, max_depth=9, oob_score=True)
# clf.fit(X_train,y_train.values.ravel())

In [14]:
import joblib
clf = joblib.load("land_random_forest.joblib")

C:\Users\fhdtr\miniconda3\envs\tree_ram_gen\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.2.1 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\fhdtr\miniconda3\envs\tree_ram_gen\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 1.2.1 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [15]:
clf.classes_

array([1, 2, 3, 4, 5], dtype=int64)

In [16]:
print_tree(clf)

Tree 0 -----------------------------------------------------------------------------------
Number of nodes:  35
Tree 1 -----------------------------------------------------------------------------------
Number of nodes:  55
Tree 2 -----------------------------------------------------------------------------------
Number of nodes:  53
Tree 3 -----------------------------------------------------------------------------------
Number of nodes:  33
Tree 4 -----------------------------------------------------------------------------------
Number of nodes:  51
Tree 5 -----------------------------------------------------------------------------------
Number of nodes:  49
Tree 6 -----------------------------------------------------------------------------------
Number of nodes:  41
Tree 7 -----------------------------------------------------------------------------------
Number of nodes:  37
Tree 8 -----------------------------------------------------------------------------------
Number of nod

In [17]:
y_pred=clf.predict(X_test)

In [18]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.9930555555555556


# TREE RAM GEN

In [19]:
BRAM_DWIDTH      = 32
BRAM_AWIDTH      = 14

TREE_DEPTH       = 10 # number of levels
LEAF_IDX         = 0
THSH_IDX         = 1
THSH_WIDTH       = 16
ATTR_IDX         = 17
ATTR_WIDTH       = 5
RCHLD_IDX        = 22
RCHLD_WIDTH      = BRAM_DWIDTH - ATTR_WIDTH - THSH_WIDTH - 1
LAST_IDX         = 1
RES_IDX          = 2
RES_WIDTH        = 16
TREE_IDX         = 18
TREE_WIDTH       = BRAM_DWIDTH - RES_WIDTH - 2

LAST_AB_IDX      = 0
AB_TREE_IDX      = 1
AB_TREE_WIDTH    = BRAM_AWIDTH

IS_INT = True
IS_CLF = True

In [20]:
N_ESTIMATORS = len(clf.estimators_)
N_DTPS = 5
PIPELINE_STAGES = 5

bin_tree_rams = [] # list of Tree RAMs
dtp_trees = list(split(clf.estimators_, N_DTPS)) # list of subset of tree (Each subset belongs to each DTP)
for subset in dtp_trees:
    bin_bram = []
    tree_bram_gen(dtree_list=subset, no_parts=PIPELINE_STAGES, tree_bram=bin_bram)
    bin_tree_rams.append(bin_bram)

In [21]:
for bin_tree_ram in bin_tree_rams:
    print(len(bin_tree_ram))

12175
12213
11977
12015
12223


In [22]:
dec_tree_rams = []
for bin_tree_ram in bin_tree_rams:
    dec_tree_ram = []
    rambin2ramdec(bin_tree_ram, PIPELINE_STAGES, dec_tree_ram)
    dec_tree_rams.append(dec_tree_ram)

In [23]:
bin_sample = sample_dec2bin(X_test) # list of samples in binary

In [24]:
# for i in range(len(bin_sample)):
#     print(i,end=' ')
#     for a in bin_sample[i]:
#         print(a, end=' ')
#     print('')

In [25]:
np.set_printoptions(suppress=True)
predicts_hw = decision_path_bram(bin_sample[0:4], dec_tree_rams)

Sample 0 ****************************************************************************
DTP 0 ****************************************************************************
Stage 0 ------------------------------------------
5 2 1098 394
9 6 3171 3003
23 2 1098 1138
24 3 2121 1962
26 idx:  26 ,result: 3
40 2 1098 415
54 5 2682 1318
62 4 3224 3374
63 1 895 722
65 2 1098 1040
67 3 2121 3196
68 2 1098 1744
69 5 2682 2636
73 idx:  73 ,result: 3
95 4 3224 1904
105 3 2121 3358
106 0 693 695
107 5 2682 1306
111 6 3171 3194
112 5 2682 1850
114 0 693 621
116 idx:  116 ,result: 3
148 4 3224 1905
154 5 2682 1332
160 1 895 711
162 1 895 1240
163 3 2121 2150
164 5 2682 2562
166 idx:  166 ,result: 3
181 2 1098 409
189 3 2121 3456
190 4 3224 3324
191 6 3171 2906
197 1 895 711
199 4 3224 3123
205 3 2121 2874
206 3 2121 2194
207 idx:  207 ,result: 3
232 2 1098 434
250 0 693 684
268 6 3171 3336
269 3 2121 3396
270 3 2121 2939
271 0 693 912
272 idx:  272 ,result: 4
281 5 2682 726
285 6 3171 3003
297 4 3224 30

6667 6 3171 3003
6681 2 1098 1138
6682 3 2121 1990
6684 idx:  6684 ,result: 3
6698 2 1098 394
6702 3 2121 3216
6703 0 693 695
6704 5 2682 1227
6708 5 2682 1941
6710 2 1098 1030
6712 idx:  6712 ,result: 3
6743 5 2682 726
6747 5 2682 1332
6755 4 3224 3034
6759 4 3224 3788
6760 0 693 706
6761 0 693 642
6763 idx:  6763 ,result: 3
6782 1 895 584
6784 3 2121 3152
6785 0 693 520
6789 3 2121 2910
6790 1 895 941
6791 5 2682 2457
6795 idx:  6795 ,result: 3
6811 5 2682 740
6817 6 3171 3003
6839 5 2682 1941
6841 5 2682 3030
6842 0 693 717
6843 1 895 808
6845 idx:  6845 ,result: 3
6858 4 3224 1904
6862 6 3171 3003
6876 5 2682 1876
6878 6 3171 3336
6879 5 2682 3075
6880 2 1098 1562
6881 4 3224 3412
6882 6 3171 3147
6886 4 3224 3274
6887 idx:  6887 ,result: 4
6907 1 895 576
6909 2 1098 723
6915 5 2682 1307
6919 0 693 551
6921 5 2682 3075
6922 4 3224 3461
6923 3 2121 2932
6924 0 693 684
6928 idx:  6928 ,result: 4
6946 4 3224 1866
6950 5 2682 1332
6956 0 693 551
6958 5 2682 3030
6959 3 2121 2352
6960 1

756 2 336 528
757 idx:  757 ,result: 0
802 1 517 573
803 idx:  803 ,result: 0
837 6 2537 2612
838 5 753 726
842 1 517 573
843 idx:  843 ,result: 0
886 1 517 588
887 idx:  887 ,result: 0
915 1 517 573
916 idx:  916 ,result: 0
952 4 1817 1880
953 2 336 534
954 idx:  954 ,result: 0
989 4 1817 1880
990 5 753 884
991 1 517 587
992 idx:  992 ,result: 0
1028 0 306 374
1029 5 753 726
1031 3 3811 3982
1032 idx:  1032 ,result: 0
1069 5 753 726
1073 2 336 762
1074 1 517 608
1075 1 517 584
1076 idx:  1076 ,result: 0
1106 0 306 372
1107 3 3811 3996
1108 idx:  1108 ,result: 0
1143 2 336 394
1144 4 1817 1930
1145 4 1817 1853
1146 idx:  1146 ,result: 0
1190 5 753 752
1200 0 306 644
1201 5 753 1332
1202 3 3811 3616
1208 0 306 352
1209 2 336 424
1210 idx:  1210 ,result: 2
1235 1 517 608
1236 5 753 1188
1237 5 753 726
1239 2 336 376
1240 0 306 306
1241 idx:  1241 ,result: 0
1276 1 517 596
1277 idx:  1277 ,result: 0
1315 6 2537 2612
1316 4 1817 1829
1317 2 336 594
1318 idx:  1318 ,result: 0
1356 0 306 372

In [26]:
for labels in predicts_hw:
    print(labels)

[0, 0, 0, 1342, 158]
[80, 1391, 21, 7, 1]
[1437, 0, 63, 0, 0]
[1500, 0, 0, 0, 0]


In [43]:
predicts_hw = [clf.classes_[labels.index(max(labels))] for labels in predicts_hw]

In [44]:
print("Accuracy:",metrics.accuracy_score(y_test, predicts_hw))

Accuracy: 0.9930555555555556


# COEFF FILE GEN

In [45]:
attribute_coeff_file_gen(X_test.values, 'land_int_sample')

In [46]:
for i in range(len(bin_tree_rams)):
    tree_coeff_file_gen(bin_tree_rams[i], 'land_int_tree_5dtp_'+str(i))

# DRAM FILE GEN

In [29]:
sample_mem_file_gen(bin_sample,'land_int_samples_mem')

In [28]:
tree_mem_file_gen(bin_tree_rams,'land_int_forest_mem_5dtp')